In [1]:
import numpy as np
import re
import scipy.stats as st
import os

In [6]:
log_path = "log_safe/minigrid_green_blue_big_mask_extract-42-mask-linear_comb/230910-105443"
n_tasks = 5

log_file_path = f"{log_path}/train-log-{log_path.split('/')[-1]}.txt"
save_conf_file_path = f"{log_path}/conf_data.csv"
save_sem_file_path = f"{log_path}/sem_data.csv"

if os.path.exists(save_conf_file_path):
    os.remove(save_conf_file_path)

if os.path.exists(save_sem_file_path):
    os.remove(save_sem_file_path)

conf_data = np.zeros(n_tasks,)
sem_data = np.zeros(n_tasks,)
task_idx = 0

with open(log_file_path, 'r') as file:
    for line in file:
        if '- [' in line:
            pattern = r'\[(.*?)\]'
            value_str = re.findall(pattern, line)[0]
            values = np.array([float(val) for val in value_str.split(',')])
            mean = np.mean(values)
            sem = st.sem(values)
            if mean > 0:
                lower_bound, _ = st.t.interval(0.95, len(values)-1, loc=mean, scale=sem)
                conf = mean - lower_bound
                conf_data[task_idx] = conf
                sem_data[task_idx] = sem
            task_idx += 1
        
        if task_idx == n_tasks:
            task_idx = 0
            with open(save_conf_file_path, 'a') as conf_file:
                np.savetxt(conf_file, conf_data.reshape(1, -1), delimiter=',', fmt='%.4f')
            with open(save_sem_file_path, 'a') as sem_file:
                np.savetxt(sem_file, sem_data.reshape(1, -1), delimiter=',', fmt='%.4f')